In [17]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
import utils.osm_helper
reload(utils.osm_helper)
from utils.osm_helper import *
import geopandas as gp
from osmread import Node, Way, Relation
import datetime
import pandas as pd

In [7]:
OSM_DC_BBOX_PATH = '../data/osm/osm_dc_bbox.osm'
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()

begin reading osm 2016-11-12 19:04:28.942000
finish reading osm 2016-11-12 19:05:40.775000
["len of <class 'osmread.elements.Relation'> = 3427", "len of <class 'osmread.elements.Node'> = 2546260", "len of <class 'osmread.elements.Way'> = 306332"]


In [8]:
DC_OSM_IDS_PATH ='../data/osm_ids_have_tag_near_opendc_segs.txt'
DC_OSM_IDS={}
with open(DC_OSM_IDS_PATH) as f:
    DC_OSM_IDS = eval(f.readlines()[0])
for key, value in DC_OSM_IDS.items():
    print key, len(value)
    
    
def dc_obj(osm_objs, osm_ids, osmtype, osmtype_str):
    return [obj for obj in osm_objs.osm_objs[osmtype] if obj.id in osm_ids[osmtype_str]]

DC_NODES = dc_obj(OSM_DC_BBOX, DC_OSM_IDS,Node, 'Node')
DC_WAYS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Way, 'Way')
DC_RLTNS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Relation, 'Relation')
len(DC_WAYS), len(DC_NODES), len(DC_RLTNS)

Node 19386
Relation 2203
Way 140629


(140629, 19386, 2203)

In [6]:
# TODO: move to osm helper.py




In [9]:
# specific filter
def filter_is_motorway(obj):
    tag_highway = [('highway','*')]
    tag_bike_walk = [('highway', set(['path','pedestrian','footway','steps','cycleway','crossing']))]
    return filter_obj(obj, tag_highway, tag_bike_walk)
def filter_isnot_motorway(obj):
    return not filter_is_motorway(obj)

def filter_is_bike_walk_way(obj):
    tag_bike_walk = [('highway', set(['path','pedestrian','footway','steps','cycleway','crossing']))]
    return filter_obj(obj, tag_bike_walk)
def filter_isnot_bike_walk_way(obj):
    return not filter_is_bike_walk_way(obj)

def filter_is_admin(obj):
    tag_admin_have = [('boundary','*'),('place','*')]
    tag_admin_donthave = [('leisure','*'), ('amenity','*'), ('boundary',['national_park','protected_area']),
                          ('natural','*'), ('place', ['island', 'islet', 'square', 'farm'])]
    return filter_obj(obj, tag_admin_have, tag_admin_donthave)

def filter_isnot_admin(obj):
    return not filter_is_admin(obj)

def filter_is_restriction(obj):
    tag_restriction = [('restriction','*'),('restriction:conditional','*'), ('restriction:hgv','*')]
    return filter_obj(obj, tag_restriction)

def filter_isnot_restriction(obj):
    return not filter_is_restriction(obj)

def filter_is_landuse(obj):
    tag_landuse = [('landuse','*')]
    return filter_obj(obj, tag_landuse)

def filter_isnot_landuse(obj):
    return not filter_is_landuse(obj)

In [190]:
# DC_NODES_WITH_TAG = filter_osm_data(DC_NODES)
# DC_WAYS_WITH_TAG = filter_osm_data(DC_WAYS)
# DC_RLTNS_WITH_TAG = filter_osm_data(DC_RLTNS)
# len(DC_NODES_WITH_TAG), len(DC_WAYS_WITH_TAG), len(DC_RLTNS_WITH_TAG)

In [10]:
restriction_rltn = filter_osm_data(DC_RLTNS,special_filters=[filter_is_restriction])
landuse_rltn = filter_osm_data(DC_RLTNS,special_filters=[filter_is_landuse])
landuse_way = filter_osm_data(DC_WAYS,special_filters=[filter_is_landuse])
admin_rltn = filter_osm_data(DC_RLTNS, special_filters=[filter_is_admin])
admin_way = filter_osm_data(DC_WAYS, special_filters=[filter_is_admin])
motor_way = filter_osm_data(DC_WAYS,special_filters=[filter_is_motorway])
len(landuse_rltn), len(landuse_way), len(admin_rltn),len(admin_way), len(restriction_rltn), len(motor_way)

(17, 369, 153, 147, 640, 12236)

In [13]:
DC_WAYS_NOT_ADMIN_MOTOR_LANDUSE = filter_osm_data(DC_WAYS,special_filters=[filter_isnot_admin, filter_isnot_motorway,filter_isnot_landuse])
print len(DC_WAYS_NOT_ADMIN_MOTOR_LANDUSE), len(DC_WAYS)-len(admin_way)-len(motor_way) - len(landuse_way)

DC_RLTNS_NOT_ADMIN_RESTRICTION_LANDUSE = filter_osm_data(DC_RLTNS,special_filters=[filter_isnot_admin, filter_isnot_restriction,filter_isnot_landuse])
len(DC_RLTNS_NOT_ADMIN_RESTRICTION_LANDUSE), len(DC_RLTNS)-len(admin_rltn) - len(restriction_rltn) - len(landuse_rltn)

127877 127877


(1394, 1393)

In [20]:
import shapely.geometry as shpgeo

def lon_lats_closed(lon_lats):
    return lon_lats[0]==lon_lats[-1]

def way2lineOrpoly(osm_data, way):
    nodes = [osm_data.get_osm_node_by_id(nid) for nid in way.nodes]
    lon_lats = [(node.lon, node.lat) for node in nodes]
    return shpgeo.Polygon(lon_lats).buffer(0) if lon_lats_closed(lon_lats) and len(lon_lats)>3 else shpgeo.LineString(lon_lats)

def flatten_rltn_shpcltn(shpcltn):
    flat_shpcltn = []
    for shpobjs in shpcltn:
        flat_shpcltn.extend(shpobjs)
    return flat_shpcltn

def build_gpdf_list_from_osm(osm_data, nodes, ways, rltns):
    columns = ['id','type','geometry']
    shpobjs_from_nodes = [(node.id,'Node', node2pt(node)) for node in nodes]
    shpobjs_from_ways =  [(way.id,'Way', way2lineOrpoly(osm_data, way)) for way in ways]
    shpobjs_from_rltns = [(rltn.id, 'Relation', shpobj) for rltn in rltns for shpobj in flatten_rltn_shpcltn(rltn2mergedCltn(osm_data,rltn)[0])]
    print len(shpobjs_from_nodes),len(shpobjs_from_ways),len(shpobjs_from_rltns)
    shpobjs = []
    shpobjs.extend(shpobjs_from_nodes)
    shpobjs.extend(shpobjs_from_ways)
    shpobjs.extend(shpobjs_from_rltns)
    gpdf = gp.GeoDataFrame(shpobjs, columns=columns)
    return gpdf

In [21]:
GPDF_NEARBY = build_gpdf_list_from_osm(OSM_DC_BBOX, DC_NODES,DC_WAYS_NOT_ADMIN_MOTOR_LANDUSE, DC_RLTNS_NOT_ADMIN_RESTRICTION_LANDUSE)

19386 127877 1888


In [26]:
GPDF_NEARBY2 = build_gpdf_list_from_osm2(OSM_DC_BBOX, DC_NODES,DC_WAYS_NOT_ADMIN_MOTOR_LANDUSE, DC_RLTNS_NOT_ADMIN_RESTRICTION_LANDUSE)

build_gpdf_list len(shpobjs_from_nodes)=19386,len(shpobjs_from_ways)=127877,len(shpobjs_from_rltns)=1888


In [27]:
def remove_equal_shpobj(objs):
    import rtree
    size = len(objs)
    equal_pair = []
    keep = []
    exclude_idx = set()
    
    tree_idx = rtree.index.Index()
    objs_bounds = [o.bounds for o in objs]
    for i in xrange(size):
        try:
            tree_idx.insert(i, objs_bounds[i])
        except Exception as e:
            print i, objs_bounds[i], objs[i]
            raise e

    for i in xrange(size):
        if i in exclude_idx:
            continue
        keep.append(i)
        js = tree_idx.intersection(objs[i].bounds)
        for j in js:
            if i!=j and objs[i].equals(objs[j]):
                equal_pair.append((i,int(j)))
                exclude_idx.add(j)
                
    return keep, equal_pair

In [28]:
def find_direct_parent(tree, node, direct_parents):
    if node in direct_parents:
        return direct_parents[node]
    node_parents = tree[tree.child==node].parent.values
    if len(node_parents)==0:
        direct_parents[node]=(node, 'root',1)
        return direct_parents[node]
    node_parents_lv = [find_direct_parent(tree, p, direct_parents)[2] for p in node_parents]
    max_lv = max(node_parents_lv)
    direct_parent = node_parents[node_parents_lv.index(max_lv)]
    direct_parents[node] = (node, direct_parent, max_lv+1)
    return (node, direct_parent, max_lv+1)

def find_tree2(messy_tree_df):
    direct_parents={}
    all_nodes = set(messy_tree_df.child)
    messy_tree_df = messy_tree_df[messy_tree_df.child!=messy_tree_df.parent].copy()
    new_tree = []
    for node in all_nodes:
        x = find_direct_parent(messy_tree_df, node, direct_parents)
        new_tree.append(x)
    return pd.DataFrame(new_tree,columns=['child','parent','lv']).sort('lv')


In [29]:
from utils.other_utils import find_tree
def merge_within(shp_gpdf):
    import geopandas as gp
    import pandas as pd
    print 'begin merge within', datetime.datetime.now()
    keep, equal_pair= remove_equal_shpobj(shp_gpdf.geometry.values)
    equal_pair_index = [(shp_gpdf.iloc[i].name, shp_gpdf.iloc[j].name) for i, j in equal_pair]
    gpdf_no_equal = shp_gpdf.iloc[keep]
    print 'keep =',len(keep), 'equal pair =',len(equal_pair), gpdf_no_equal.shape, datetime.datetime.now()
    sjoin = gp.tools.sjoin(gpdf_no_equal,gpdf_no_equal,op='within')
    print 'sjoin.shape =',sjoin.shape, datetime.datetime.now()
    messy_tree_df = pd.DataFrame(zip(sjoin.index.values, sjoin.index_right.values), columns=['child','parent'])
    print 'messy tree shape =', messy_tree_df.shape, datetime.datetime.now()
    clean_tree_df = find_tree2(messy_tree_df.copy())
    print 'clean tree shape =', clean_tree_df.shape, datetime.datetime.now()
    top_level_shp_idx = clean_tree_df[clean_tree_df.parent=='root'].child.values
    return shp_gpdf[shp_gpdf.index.isin(top_level_shp_idx)], messy_tree_df, clean_tree_df, equal_pair_index

In [193]:
GPDF_NEARBY_MEGED, MERGED_HISTORY, MERGE_HIERARCHY, EQ_PAIR_INDEX = merge_within(GPDF_NEARBY)

begin merge within 2016-11-11 12:06:03.837000
keep = 148430 equal pair = 721 (148430, 3) 2016-11-11 12:07:02.736000
sjoin.shape = (166597, 6) 2016-11-11 12:12:24.083000
messy tree shape = (166597, 2) 2016-11-11 12:12:24.200000
clean tree shape = (148430, 3) 2016-11-11 12:14:50.358000


In [195]:
EQUAL_PAIR = [(GPDF_NEARBY.loc[i].id,GPDF_NEARBY.loc[i]['type'], GPDF_NEARBY.loc[j].id, GPDF_NEARBY.loc[j]['type']) for i, j in EQ_PAIR_INDEX]

In [200]:
pd.DataFrame(EQUAL_PAIR,columns=['child_oid','child_otype','parent_oid','parent_otype']).to_csv('../data/osm_equal_pair.csv')

In [202]:
temp_df = MERGE_HIERARCHY.merge(GPDF_NEARBY[['id','type']], how='left', left_on='child', right_index=True).merge(GPDF_NEARBY[['id','type']], how='left',left_on='parent', right_index=True)
temp_df.columns=['child','parent','lv','child_oid', 'child_otype', 'parent_oid','parent_otype']
temp_df.parent_oid = temp_df.parent_oid.fillna('')
temp_df.to_csv('../data/osm_child_direct_parent.csv')

In [198]:
temp_df = MERGED_HISTORY.merge(GPDF_NEARBY[['id','type']], how='left', left_on='child', right_index=True).merge(GPDF_NEARBY[['id','type']], how='left',left_on='parent', right_index=True)
temp_df.columns=['child','parent','child_oid', 'child_otype', 'parent_oid','parent_otype']
temp_df.to_csv('../data/osm_child_all_parents.csv')

In [ ]:
temp_df[temp_df.child==22941]

In [61]:
temp_df[temp_df.parent==164426].shape, temp_df[temp_df.parent==22941].shape

((725, 6), (96, 6))

In [69]:
temp_df[temp_df.parent==157636].type_x.value_counts()

Way    69
dtype: int64

In [76]:
for way_id in temp_df[temp_df.id_y==67383293].id_x.values:
    way = OSM_DC_BBOX.get_osm_way_by_id(way_id)
    print way.tags

{u'source': u'dcgis', u'landuse': u'residential', u'dcgis:pubdate': u'20090505', u'dataset': u'PublicHousingAreaPly'}


In [125]:
parent_cnt = temp_df.parent.value_counts()
for p in parent_cnt[parent_cnt>100].index.tolist():
    oid = temp_df[(temp_df.child==p) & (temp_df.parent==p)].id_x.values[0]
    otype = temp_df[(temp_df.child==p) & (temp_df.parent==p)].type_x.values[0]
    print 'http://www.openstreetmap.org/{}/{}'.format(otype.lower(), oid), temp_df[temp_df.parent==p].shape
#     osmtype = {'Way':Way, 'Node':Node, 'Relation':Relation}[otype]
#     obj = OSM_DC_BBOX.get_osm_obj_by_id(osmtype, oid)
#     if have_tag_value(obj, 'landuse', '*'):
#         print 'http://www.openstreetmap.org/{}/{}'.format(otype.lower(), oid), temp_df[temp_df.parent==p].shape, obj.tags['landuse']

http://www.openstreetmap.org/way/296355721 (1524, 6)
http://www.openstreetmap.org/way/440070008 (1467, 6)
http://www.openstreetmap.org/way/91525299 (1176, 6)
http://www.openstreetmap.org/relation/4610875 (725, 6)
http://www.openstreetmap.org/relation/4626530 (595, 6)
http://www.openstreetmap.org/way/132637346 (461, 6)
http://www.openstreetmap.org/way/66678705 (440, 6)
http://www.openstreetmap.org/way/440041211 (386, 6)
http://www.openstreetmap.org/way/41965038 (334, 6)
http://www.openstreetmap.org/relation/5991609 (325, 6)
http://www.openstreetmap.org/way/41964871 (289, 6)
http://www.openstreetmap.org/relation/1043453 (272, 6)
http://www.openstreetmap.org/relation/5149106 (264, 6)
http://www.openstreetmap.org/way/67383283 (262, 6)
http://www.openstreetmap.org/way/67282581 (255, 6)
http://www.openstreetmap.org/way/67282582 (239, 6)
http://www.openstreetmap.org/way/133333756 (221, 6)
http://www.openstreetmap.org/way/67282580 (193, 6)
http://www.openstreetmap.org/way/67282586 (175, 6)
htt